In [0]:
%sql
create table if not exists main.ecommerce.dim_product
(
product_id STRING,
product_category_name STRING,
product_name_lenght STRING,
product_description_lenght STRING,
product_photos_qty STRING,
product_weight_g STRING,
product_length_cm STRING,
product_height_cm STRING,
product_width_cm STRING,

effective_start_date DATE,
effective_end_date DATE,
is_Active STRING
)
USING DELTA;

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from delta.tables import DeltaTable


product_table = spark.read.table("main.ecommerce.products")

product_scd = product_table\
    .withColumn("Is_Active", lit("Y"))\
    .withColumn("effective_start_date", lit("2025-12-24"))\
    .withColumn("effective_end_date", lit("9999-12-31"))
display(product_scd)

# Existing product Delta Table
dim_products = DeltaTable.forName(spark, "main.ecommerce.dim_product")

# Create Product SCD
merge_condition = """
    t.product_id = s.product_id
    AND t.is_active = 'Y'
    """

# SCD-2 Merge Product
dim_products.alias("t").merge(
    product_scd.alias("s"),merge_condition
).whenMatchedUpdate(
    condition ="""
        t.product_category_name <> s.product_category_name OR
        t.product_name_lenght <> s.product_name_lenght OR
        t.product_description_lenght <> s.product_description_lenght OR
        t.product_photos_qty <> s.product_photos_qty OR
        t.product_weight_g <> s.product_weight_g OR
        t.product_length_cm <> s.product_length_cm OR
        t.product_height_cm <> s.product_height_cm OR
        t.product_width_cm <> s.product_width_cm
    """,
    set={
        "effective_end_date": "current_date()",
        "is_active": "'N'"
    }
).whenNotMatchedInsert(
    values={
        "product_id": "s.product_id",
        "product_category_name": "s.product_category_name",
        "product_name_lenght": "s.product_name_lenght",
        "product_description_lenght": "s.product_description_lenght",
        "product_photos_qty": "s.product_photos_qty",
        "product_weight_g": "s.product_weight_g",
        "product_length_cm": "s.product_length_cm",
        "product_height_cm": "s.product_height_cm",
        "product_width_cm": "s.product_width_cm",
        "effective_start_date": "s.effective_start_date",
        "effective_end_date": "s.effective_end_date",
        "is_active": "s.is_active"
    }
).execute()